In [211]:
# Set up the initial V-values, you can do 0 or random
import numpy as np
V = np.zeros([3,4])
V = np. random.rand(3,4)
print V

# Set up Rewards
R = np.zeros([3,4])
R[0,3] = 1
R[1,3] = -1
print R

# Set up the enter, exit cells, as well as inaccessible cells, all of these are denoted by a zero in this matrix
PI_bool = np.array([[1,1,1,0],[1,0,1,0],[1,1,1,1]])
print PI_bool



[[ 0.28932586  0.07612785  0.86436043  0.30766699]
 [ 0.58303943  0.41929991  0.03794535  0.45105288]
 [ 0.16696255  0.59824931  0.59536509  0.76882215]]
[[ 0.  0.  0.  1.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  0.  0.]]
[[1 1 1 0]
 [1 0 1 0]
 [1 1 1 1]]


In [212]:
# different policies defined here

# 2 (i or j) x 4 (right, left, up, down) x 3 (state's i position) x 4 (state's j position)

PI = np.zeros([2,4,3,4], dtype=int)

## j
PI[1,0,:,:] = np.array([[0,0,0,0],[1,1,1,1],[2,2,2,2]])
PI[1,1,:,:] = np.array([[0,0,0,0],[1,1,1,1],[2,2,2,2]])
PI[1,2,:,:] = np.array([[0,0,0,0],[0,0,0,0],[1,2,1,1]])
PI[1,3,:,:] = np.array([[1,0,1,0],[2,0,2,0],[2,2,2,2]])

## ii
PI[0,0,:,:] = np.array([[1,2,3,0],[0,0,3,0],[1,2,3,3]])
PI[0,1,:,:] = np.array([[0,0,1,0],[0,0,2,0],[0,0,1,2]])
PI[0,2,:,:] = np.array([[0,1,2,3],[0,1,2,3],[0,1,2,3]])
PI[0,3,:,:] = np.array([[0,1,2,3],[0,1,2,3],[0,1,2,3]])
# PI[0,0,:,:] = np.array([[1,2,3,np.nan],[0,np.nan,3,np.nan],[1,2,3,3]])
# PI[0,1,:,:] = np.array([[0,0,1,np.nan],[0,np.nan,2,np.nan],[0,0,1,2]])
# PI[0,2,:,:] = np.array([[0,0,0,np.nan],[0,np.nan,0,np.nan],[1,2,1,1]])
# PI[0,3,:,:] = np.array([[1,0,1,np.nan],[2,np.nan,2,np.nan],[2,2,2,2]])

print PI


[[[[1 2 3 0]
   [0 0 3 0]
   [1 2 3 3]]

  [[0 0 1 0]
   [0 0 2 0]
   [0 0 1 2]]

  [[0 1 2 3]
   [0 1 2 3]
   [0 1 2 3]]

  [[0 1 2 3]
   [0 1 2 3]
   [0 1 2 3]]]


 [[[0 0 0 0]
   [1 1 1 1]
   [2 2 2 2]]

  [[0 0 0 0]
   [1 1 1 1]
   [2 2 2 2]]

  [[0 0 0 0]
   [0 0 0 0]
   [1 2 1 1]]

  [[1 0 1 0]
   [2 0 2 0]
   [2 2 2 2]]]]


### Let's have a look how a single policy update looks like


In [214]:
# print PI[0,0,:,:]
# print PI[1,0,:,:]

print 'RIGHT policy reward'
print R[PI[1,0,:,:], PI[0,0,:,:]] * PI_bool
print 'LEFT policy reward'
print R[PI[1,1,:,:], PI[0,1,:,:]] * PI_bool
print 'UP policy reward'
print R[PI[1,2,:,:], PI[0,2,:,:]] * PI_bool
print 'DOWN policy reward'
print R[PI[1,3,:,:], PI[0,3,:,:]] * PI_bool


RIGHT policy reward
[[ 0.  0.  1.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]]
LEFT policy reward
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
UP policy reward
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]]
DOWN policy reward
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [215]:
print 'RIGHT policy value'
print V[PI[1,0,:,:], PI[0,0,:,:]] * PI_bool
print 'LEFT policy value'
print V[PI[1,1,:,:], PI[0,1,:,:]] * PI_bool
print 'UP policy value'
print V[PI[1,2,:,:], PI[0,2,:,:]] * PI_bool
print 'DOWN policy value'
print V[PI[1,3,:,:], PI[0,3,:,:]] * PI_bool

RIGHT policy value
[[ 0.07612785  0.86436043  0.30766699  0.        ]
 [ 0.58303943  0.          0.45105288  0.        ]
 [ 0.59824931  0.59536509  0.76882215  0.76882215]]
LEFT policy value
[[ 0.28932586  0.28932586  0.07612785  0.        ]
 [ 0.58303943  0.          0.03794535  0.        ]
 [ 0.16696255  0.16696255  0.59824931  0.59536509]]
UP policy value
[[ 0.28932586  0.07612785  0.86436043  0.        ]
 [ 0.28932586  0.          0.86436043  0.        ]
 [ 0.58303943  0.59824931  0.03794535  0.45105288]]
DOWN policy value
[[ 0.58303943  0.07612785  0.03794535  0.        ]
 [ 0.16696255  0.          0.59536509  0.        ]
 [ 0.16696255  0.59824931  0.59536509  0.76882215]]


### Define the value update functions

In [153]:
def get_steps(R, PI, gamma = 1):
    #gamma = 1
    # just a right step
    RS = (R[PI[1,0,:,:], PI[0,0,:,:]] * PI_bool + gamma * V[PI[1,0,:,:], PI[0,0,:,:]] * PI_bool)
    #print RS
    # just a left step
    LS = (R[PI[1,1,:,:], PI[0,1,:,:]] * PI_bool + gamma * V[PI[1,1,:,:], PI[0,1,:,:]] * PI_bool)
    #print LS
    # just an up step
    US = (R[PI[1,2,:,:], PI[0,2,:,:]] * PI_bool + gamma * V[PI[1,2,:,:], PI[0,2,:,:]] * PI_bool)
    #print US
    # just a down step
    DS = (R[PI[1,3,:,:], PI[0,3,:,:]] * PI_bool + gamma * V[PI[1,3,:,:], PI[0,3,:,:]] * PI_bool)
    #print DS
    return RS, LS, US, DS
RS, LS, US, DS = get_steps(R, PI, gamma = 1)

In [184]:
def update_V(RS, LS, US, DS):
    # A right policy update consists of 0.8*(RIGHT step) + 0.1*(UP step) + 0.1*(DOWN step)
    VR = 0.8*RS+0.1*US+0.1*DS
    #print VR
    VL = 0.8*LS+0.1*US+0.1*DS
    #print VL
    VU = 0.8*US+0.1*LS+0.1*RS
    #print VU
    VD = 0.8*DS+0.1*LS+0.1*RS
    #print VD
    #print 'maximum'
    V_new = np.array([VR, VL, VU, VD]).max(0)
    PI_new =  np.array([VR, VL, VU, VD]).argmax(0)
    return VR, VL, VU, VD, V_new, PI_new
VR, VL, VU, VD, V_new, PI_new = update_V(RS, LS, US, DS)

### Run an actual algorithm for Value Iteration

don't care about the policy, only care about the last policy

In [217]:
V = np.zeros([3,4])
for i in range(100):
#     print i
    RS, LS, US, DS = get_steps(R, PI, gamma = 0.8)
    VR, VL, VU, VD, V_new, PI_new = update_V(RS, LS, US, DS)
    V = V_new
print 'Right'
print VR
print 'Left'
print VL
print 'Up'
print VU
print 'Down'
print VD
print 'Max Value'
print V_new
print 'Optimal Policy'
print PI_new
    
print '0 - right, 1 - left, 2 - up, 3 - down'

Right
[[ 0.51965897  0.69751844  0.91549296  0.        ]
 [ 0.31897867  0.         -0.69693632  0.        ]
 [ 0.23747065  0.28404042  0.16849095  0.00846483]]
Left
[[ 0.40582891  0.44418469  0.56190476  0.        ]
 [ 0.31897867  0.          0.44109185  0.        ]
 [ 0.24776746  0.23752914  0.25386363  0.1506456 ]]
Up
[[ 0.42995594  0.56122396  0.74171697  0.        ]
 [ 0.39593065  0.          0.52816901  0.        ]
 [ 0.30012918  0.23562045  0.37280305 -0.75812411]]
Down
[[ 0.35076981  0.56122396  0.49382964  0.        ]
 [ 0.25543158  0.          0.18084747  0.        ]
 [ 0.23881624  0.23562045  0.27336883  0.13828908]]
Max Value
[[ 0.51965897  0.69751844  0.91549296  0.        ]
 [ 0.39593065  0.          0.52816901  0.        ]
 [ 0.30012918  0.28404042  0.37280305  0.1506456 ]]
Optimal Policy
[[0 0 0 0]
 [2 0 2 0]
 [2 0 2 1]]
0 - right, 1 - left, 2 - up, 3 - down
